In [29]:
import pandas as pd
import numpy as np
import math
from collections import Counter

In [2]:
path = r'C:\Users\utilizador\Documents\a_hjoaquim\a_semestre2-5oano\PSn\pratica\guia-5\german_credit_data.csv'
df = pd.read_csv(path)

In [3]:
# Our class, i.e., what we want to predict is "Purpose"
df.drop(df.columns[0],1,inplace=True)
df

,Age,Sex,Job,Housing,SavingAccounts,CheckingAccount,CreditAmount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,education
3,45,male,2,free,little,little,7882,42,furniture/equipment
4,53,male,2,free,little,little,4870,24,car
...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment
996,40,male,3,own,little,little,3857,30,car
997,38,male,2,own,little,NaN,804,12,radio/TV
998,23,male,2,free,little,little,1845,45,radio/TV


In [4]:
# Let's see our class distribution
df['Purpose'].value_counts()

car                    337
radio/TV               280
furniture/equipment    181
business                97
education               59
repairs                 22
domestic appliances     12
vacation/others         12
Name: Purpose, dtype: int64

In [5]:
# Let's make this distribution more even.
# In this example our class will have only 3 values: car, radio/TV and other
# We can thought of this re-arrangig as if the main thing we want to enphasise
# is car and radio/TV class values.

df['Purpose'] = df['Purpose'].apply(lambda x: 'other' if(x == 'furniture/equipment' or x == 'business' or x == 'education' or x == 'repairs' or x == 'domestic appliances' or x == 'vacation/others') else x)

In [6]:
# Our new distribution

df['Purpose'].value_counts()

other       383
car         337
radio/TV    280
Name: Purpose, dtype: int64

In [7]:
df.head(10)

,Age,Sex,Job,Housing,SavingAccounts,CheckingAccount,CreditAmount,Duration,Purpose
0,67,male,2,own,NaN,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,NaN,2096,12,other
3,45,male,2,free,little,little,7882,42,other
4,53,male,2,free,little,little,4870,24,car
5,35,male,1,free,NaN,NaN,9055,36,other
6,53,male,2,own,quite rich,NaN,2835,24,other
7,35,male,3,rent,little,moderate,6948,36,car
8,61,male,1,own,rich,NaN,3059,12,radio/TV
9,28,male,3,own,little,moderate,5234,30,car


In [8]:
# Since we've discrete and continuous variables,
# let's build a dict where we'll store the correspondence
# between the column and the it's type.

cont = 'Continuous'; disc = 'Discrete'
dataDescription = { 'Age': cont, 'Sex': disc, 'Housing': disc, 'SavingAccounts': disc, 'CheckingAccount': disc, 'CheckingAccount': disc, 'CreditAmount': cont, 'Duration': disc, 'Purpose': disc }
dataDescription

{'Age': 'Continuous',
 'Sex': 'Discrete',
 'Housing': 'Discrete',
 'SavingAccounts': 'Discrete',
 'CheckingAccount': 'Discrete',
 'CreditAmount': 'Continuous',
 'Duration': 'Discrete',
 'Purpose': 'Discrete'}

In [20]:
# Let's build a function to do the above mentioned.
# It will receive a dict with the data description and the dataset itself
# Then detect wich column(s) need treatment and "remove" the missing values accordingly:
# For discrete attributes: most common value method
# For continuous attributes: medium value method
# Returns the dataset without missing values.
# Note: this is a pure function, i.e., we don't directly mess with the original dataset.

def removeNaN(df, dataDescription):
    wCopy = df.copy()
    attributesWnan = []

    for key in dataDescription:
        if wCopy[key].isnull().values.any() == True:
            attributesWnan.append(key)

    for key in attributesWnan:
        if(dataDescription[key] == 'Discrete'):
            wCopy[key] = wCopy[key].fillna(wCopy[key].value_counts().idxmax())
        else:
            wCopy[key] = wCopy[key].fillna(wCopy[key].mean)
    
    return wCopy

In [24]:
# In this step we should alredy see that our dataframe don't have any missing values.

dfWithoutNan = removeNaN(df, dataDescription)
dfWithoutNan.head(10)

,Age,Sex,Job,Housing,SavingAccounts,CheckingAccount,CreditAmount,Duration,Purpose
0,67,male,2,own,little,little,1169,6,radio/TV
1,22,female,2,own,little,moderate,5951,48,radio/TV
2,49,male,1,own,little,little,2096,12,other
3,45,male,2,free,little,little,7882,42,other
4,53,male,2,free,little,little,4870,24,car
5,35,male,1,free,little,little,9055,36,other
6,53,male,2,own,quite rich,little,2835,24,other
7,35,male,3,rent,little,moderate,6948,36,car
8,61,male,1,own,rich,little,3059,12,radio/TV
9,28,male,3,own,little,moderate,5234,30,car


In [52]:
# data should be a list of probabilities
def gini(probs):
    summ = 0
    for key in probs:
        summ = summ + key**2
    return(1-summ)

val = [0.25,0.25,0.25,0.25]
gini(val)

0.75

In [53]:
# data should be a list of probabilities
def entropy(probs):
    summ = 0
    for key in probs:
        summ = summ + key * math.log(key,2)
    return(-summ)

val = [0.25,0.25,0.25,0.25]
entropy(val)

2.0

In [56]:
# df: dataframe to process
# var: dataframe variables to use as predictors
# classe: class we want to predict 
# ef: impurity function we want to use
# Example of use-case: DT = CalcDT(data,data.columns.drop('Purpose'),'Purpose',Entropy)

def calcDT(df, var, classe, ef):
    print("Hello world")
    
calcDt(1,2,3,4)

Hello world
